In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.inter_rater import fleiss_kappa
import simpledorff  # per Krippendorff come prima

In [2]:
file_name = "../annotated_multipico_new.xlsx"
sheet_names = ['Alessandro', 'Cristina', 'Eliana', 'Michele', 'Michael', 'Elisa']
df = pd.read_excel(file_name, sheet_name=sheet_names)
df['Alessandro'].head()

,level,source,reply,post,language,language_variety,reply_id,post_id,majority_vote,Unnamed: 9,rhetorical figure,comment
0,2,twitter,@USER @USER Anche per rompere il ghiaccio,@USER Gianca già che ci sei regalale un paio d...,it,it,15902900000000000000,7961696982295378944,1,NaN,"CONTEXT SHIFT, HYPERBOLE",NaN
1,2,twitter,@USER Nonnino! Già sveglio? ??,@USER Non troppo buongiorno,it,it,4652170000000000000,4824587311292975104,1,NaN,"RHETORICAL QUESTION, HYPERBOLE",NaN
2,1,twitter,@USER il terrone di essere rimpiazzata,il terrore di essere rimpiazzata,it,it,18161900000000000000,7155693771356047360,1,NaN,"CONTEXT SHIFT, OTHER",NaN
3,1,twitter,@USER Tanto per non essere banali,"Elly #Schlein: ""Il lavoro deve tornare a esser...",it,it,5855670000000000000,5929177359346716672,1,NaN,FALSE ASSERTION,NaN
4,1,reddit,Se le toghe rosse avessere mandato un avviso d...,Se solo la società perseguisse l'obiettivo di ...,it,it,10284800000000000000,12099038560420169728,1,NaN,"OXYMORON, ANALOGY",NaN


In [3]:
file_name = "../annotated_multipico_new.xlsx"
sheet_names = ['Alessandro', 'Cristina', 'Eliana', 'Michele', 'Michael', 'Elisa']

# Carico solo il foglio di Cristina
df_cristina = pd.read_excel(file_name, sheet_name='Cristina')
colonna_rf = df_cristina['rhetorical figure']

# Trovo l'indice dell'ultima riga non vuota nella colonna 'rhetorical figure' di Cristina
ultima_riga_non_vuota = colonna_rf.last_valid_index()
print("L'ultima riga annotata da Cristina (per la colonna rhetorical figure) è:", ultima_riga_non_vuota)

# Ora carico tutti gli sheet
df_dict = pd.read_excel(file_name, sheet_name=sheet_names)

# Taglio ogni DataFrame fino all'ultima riga non vuota annotata da Cristina
# ATTENZIONE: last_valid_index() è l'indice dell'ultima riga valida, potrebbe essere un numero qualsiasi,
# bisogna verificare se è un indice numerico o un indice di riga (se è un RangeIndex tipicamente lo è)
# Assumiamo che sia un indice numerico, quindi per prendere le prime righe useremo .iloc[:ultima_riga_non_vuota + 1]

df_dict_truncated = {}
for sheet, df in df_dict.items():
    df_dict_truncated[sheet] = df.iloc[:ultima_riga_non_vuota + 1]

# Ora df_dict_truncated contiene tutti i DataFrame tagliati al numero di righe di Cristina
df = df_dict_truncated

L'ultima riga annotata da Cristina (per la colonna rhetorical figure) è: 200


In [4]:
# DROP colonne indesiderate subito dopo il caricamento
cols_to_drop = ['level', 'source', 'language', 'language_variety', 'majority_vote', 'Unnamed: 9', 'comment']
for name in df:
    df[name] = df[name].drop(columns=[col for col in cols_to_drop if col in df[name].columns])
    # Aggiungi la colonna 'indice' con il valore dell'indice di ogni riga
    df[name]['index'] = df[name].index
df['Alessandro'].head()

,reply,post,reply_id,post_id,rhetorical figure,index
0,@USER @USER Anche per rompere il ghiaccio,@USER Gianca già che ci sei regalale un paio d...,15902900000000000000,7961696982295378944,"CONTEXT SHIFT, HYPERBOLE",0
1,@USER Nonnino! Già sveglio? ??,@USER Non troppo buongiorno,4652170000000000000,4824587311292975104,"RHETORICAL QUESTION, HYPERBOLE",1
2,@USER il terrone di essere rimpiazzata,il terrore di essere rimpiazzata,18161900000000000000,7155693771356047360,"CONTEXT SHIFT, OTHER",2
3,@USER Tanto per non essere banali,"Elly #Schlein: ""Il lavoro deve tornare a esser...",5855670000000000000,5929177359346716672,FALSE ASSERTION,3
4,Se le toghe rosse avessere mandato un avviso d...,Se solo la società perseguisse l'obiettivo di ...,10284800000000000000,12099038560420169728,"OXYMORON, ANALOGY",4


In [5]:
# Per ogni foglio, tieni solo righe con 'rhetorical figure' non-NaN
'''cleaned = {
    name: sheet[sheet['rhetorical figure'].notna()][['post_id', 'reply_id']]
    for name, sheet in df.items()
}

sets_of_pairs = [set(zip(sheet['post_id'], sheet['reply_id'])) for sheet in cleaned.values()]
common_pairs = set.intersection(*sets_of_pairs)
print(f"Coppie (post_id, reply_id) comuni con 'rhetorical figure' non-NaN in tutti i fogli: {len(common_pairs)}")

final_cleaned = {}
for name, sheet in df.items():
    mask = sheet.apply(lambda row: (row['post_id'], row['reply_id']) in common_pairs, axis=1)
    mask &= sheet['rhetorical figure'].notna()
    final_cleaned[name] = sheet[mask].copy()

for sheet_name in sheet_names:
    print(f"Numero di post in {sheet_name}: {len(final_cleaned[sheet_name])}")

final_cleaned['Alessandro'].head()'''
final_cleaned = df
final_cleaned['Michele'].iloc[27:32]

,reply,post,reply_id,post_id,rhetorical figure,index
27,Se po buccà chi drento?,Siamo sicuri non sia matteo montesi?,10540900000000000000,1004779540710112000,OTHER,27
28,@USER @USER Tarlo mentale.,@USER @USER Per esiliare dal suo amico sceicc...,6975850000000000000,10742053278056030208,NaN,28
29,@USER @USER La giustizia e' aliena su questo ...,@USER @USER Sì ma non è Giusto.,15962200000000000000,14374185223286280192,HYPERBOLE,29
30,Perché avere delle bottiglie di vino a casa qu...,Io non ho mai capito perché si dovrebbero salv...,5036060000000000000,16167343742615099392,RHETORICAL QUESTION,30
31,@USER I don’t,@USER Non essere così cattivo con lei...❤️❤️❤️,16985200000000000000,4320113204215145984,NaN,31


In [6]:
len(final_cleaned['Michele'])

201

In [7]:
# Estrai tutte le figure retoriche da tutti i fogli, normalizzate
all_figures_set = set()

def normalize_figures(cell):
    # Gestione NaN
    if pd.isna(cell):
        return []
    figures = [f.strip().upper() for f in str(cell).split(',')]
    # Sostituzione specifica
    figures = ["OXYMORON" if f == "OXYMORON/PARADOX" else f for f in figures]
    figures = ["EUPHEMISM" if f == "EUPHEMINSM" else f for f in figures]
    return figures

# Raccogli tutte le figure da tutti i fogli
for sheet in final_cleaned.values():
    for entry in sheet['rhetorical figure'].dropna():
        all_figures_set.update(normalize_figures(entry))

# Ordina per consistenza
all_figures = sorted(all_figures_set)

# Aggiungi una colonna per ciascuna figura: 1 se presente, 0 altrimenti
for name, sheet in final_cleaned.items():
    for fig in all_figures:
        sheet[fig] = sheet['rhetorical figure'].apply(
            lambda x: 1 if fig in normalize_figures(x) else 0
        )
    final_cleaned[name] = sheet  # Aggiorna

final_cleaned['Alessandro'].head()

,reply,post,reply_id,post_id,rhetorical figure,index,ANALOGY,CONTEXT SHIFT,EUPHEMISM,FALSE ASSERTION,HYPERBOLE,OTHER,OXYMORON,RHETORICAL QUESTION
0,@USER @USER Anche per rompere il ghiaccio,@USER Gianca già che ci sei regalale un paio d...,15902900000000000000,7961696982295378944,"CONTEXT SHIFT, HYPERBOLE",0,0,1,0,0,1,0,0,0
1,@USER Nonnino! Già sveglio? ??,@USER Non troppo buongiorno,4652170000000000000,4824587311292975104,"RHETORICAL QUESTION, HYPERBOLE",1,0,0,0,0,1,0,0,1
2,@USER il terrone di essere rimpiazzata,il terrore di essere rimpiazzata,18161900000000000000,7155693771356047360,"CONTEXT SHIFT, OTHER",2,0,1,0,0,0,1,0,0
3,@USER Tanto per non essere banali,"Elly #Schlein: ""Il lavoro deve tornare a esser...",5855670000000000000,5929177359346716672,FALSE ASSERTION,3,0,0,0,1,0,0,0,0
4,Se le toghe rosse avessere mandato un avviso d...,Se solo la società perseguisse l'obiettivo di ...,10284800000000000000,12099038560420169728,"OXYMORON, ANALOGY",4,1,0,0,0,0,0,1,0


## Complessivo: Estrazione etichetta più frequente per ogni record

In [8]:
import pandas as pd

# Numero annotatori e lista annotatori
num_annotators = len(sheet_names)
print(f"Numero di annotatori: {num_annotators}")

# Prepara i dataframe indicizzati
dfs_figs = []
for name in sheet_names:
    df_tmp = final_cleaned[name][['post_id', 'reply_id', 'index'] + all_figures].copy()
    dfs_figs.append(df_tmp.set_index(['post_id', 'reply_id', 'index']))

# Trova record comuni
common_index = dfs_figs[0].index
for df_tmp in dfs_figs[1:]:
    common_index = common_index.intersection(df_tmp.index)

results = []

for idx in common_index:
    # Matrice annotatori x figure per il record
    matrix = pd.DataFrame([df.loc[idx].values for df in dfs_figs], columns=all_figures)

    # Conta annotatori per figura
    votes_per_figure = matrix.sum(axis=0)
    print(f"Record {idx}: voti per figura: {votes_per_figure.to_dict()}")
    
    max_votes = votes_per_figure.max()  # Se c'è un pareggio prende casualmente la prima in ordine alfabetico
    # Trova figure con max voti
    top_figures = votes_per_figure[votes_per_figure == max_votes].index.tolist()
    
    # Scegli la prima in ordine alfabetico
    top_label = sorted(top_figures)[0] if top_figures else None
    print(f"Record {idx}: figura con max voti: {top_label} ({max_votes} voti)")
    

    results.append({
        'index': idx[2],
        'post_id': idx[0],
        'reply_id': idx[1],
        'most_frequent_label': top_label,
        'votes': max_votes
    })
    
    print('\n')

df_most_frequent = pd.DataFrame(results)
print(len(df_most_frequent))


Numero di annotatori: 6
Record (7961696982295378944, 15902900000000000000, 0): voti per figura: {'ANALOGY': 0, 'CONTEXT SHIFT': 5, 'EUPHEMISM': 1, 'FALSE ASSERTION': 1, 'HYPERBOLE': 3, 'OTHER': 0, 'OXYMORON': 0, 'RHETORICAL QUESTION': 0}
Record (7961696982295378944, 15902900000000000000, 0): figura con max voti: CONTEXT SHIFT (5 voti)


Record (4824587311292975104, 4652170000000000000, 1): voti per figura: {'ANALOGY': 0, 'CONTEXT SHIFT': 0, 'EUPHEMISM': 1, 'FALSE ASSERTION': 0, 'HYPERBOLE': 1, 'OTHER': 0, 'OXYMORON': 2, 'RHETORICAL QUESTION': 5}
Record (4824587311292975104, 4652170000000000000, 1): figura con max voti: RHETORICAL QUESTION (5 voti)


Record (7155693771356047360, 18161900000000000000, 2): voti per figura: {'ANALOGY': 0, 'CONTEXT SHIFT': 2, 'EUPHEMISM': 0, 'FALSE ASSERTION': 1, 'HYPERBOLE': 0, 'OTHER': 5, 'OXYMORON': 0, 'RHETORICAL QUESTION': 0}
Record (7155693771356047360, 18161900000000000000, 2): figura con max voti: OTHER (5 voti)


Record (5929177359346716672, 585567

Record (12561098575626790912, 12674400000000000000, 93): voti per figura: {'ANALOGY': 0, 'CONTEXT SHIFT': 1, 'EUPHEMISM': 4, 'FALSE ASSERTION': 2, 'HYPERBOLE': 0, 'OTHER': 1, 'OXYMORON': 0, 'RHETORICAL QUESTION': 0}
Record (12561098575626790912, 12674400000000000000, 93): figura con max voti: EUPHEMISM (4 voti)


Record (12802596261741490176, 6395060000000000000, 94): voti per figura: {'ANALOGY': 0, 'CONTEXT SHIFT': 3, 'EUPHEMISM': 1, 'FALSE ASSERTION': 4, 'HYPERBOLE': 0, 'OTHER': 0, 'OXYMORON': 0, 'RHETORICAL QUESTION': 0}
Record (12802596261741490176, 6395060000000000000, 94): figura con max voti: FALSE ASSERTION (4 voti)


Record (11534803039351519232, 14401400000000000000, 95): voti per figura: {'ANALOGY': 0, 'CONTEXT SHIFT': 0, 'EUPHEMISM': 0, 'FALSE ASSERTION': 0, 'HYPERBOLE': 0, 'OTHER': 5, 'OXYMORON': 0, 'RHETORICAL QUESTION': 0}
Record (11534803039351519232, 14401400000000000000, 95): figura con max voti: OTHER (5 voti)


Record (1236265448438971904, 4088920000000000000, 96): 

In [9]:
# Rinominare la colonna 'most_frequent_label' in 'actual'
df_actual = df_most_frequent.rename(columns={'most_frequent_label': 'actual'})

# Selezionare solo le colonne desiderate
df_actual = df_actual[['index', 'post_id', 'reply_id', 'actual']]

# Salvare su file CSV
df_actual.to_csv('actuals/global_actual_labels.csv', index=False)

print("File 'actual_labels.csv' creato con successo.")


File 'actual_labels.csv' creato con successo.


## Genere: Estrazione etichetta più frequente per ogni record

In [10]:
import pandas as pd

# Lista annotatori e gruppo per genere (modifica se necessario)
male_annotators = ["Alessandro", "Michael", "Michele"]
female_annotators = ["Cristina", "Elisa", "Eliana"]

# Prepara i dataframe indicizzati per ogni annotatore
dfs_figs = {}
for name in sheet_names:
    df_tmp = final_cleaned[name][['post_id', 'reply_id', 'index'] + all_figures].copy()
    dfs_figs[name] = df_tmp.set_index(['post_id', 'reply_id', 'index'])

# Trova record comuni a tutti
common_index = set(dfs_figs[male_annotators[0]].index)
for name in sheet_names[1:]:
    common_index = common_index.intersection(dfs_figs[name].index)
common_index = list(common_index)

results = []

for idx in common_index:
    # Per gruppo maschile: prendi righe solo di annotatori maschi
    male_matrix = pd.DataFrame([dfs_figs[name].loc[idx].values for name in male_annotators], columns=all_figures)
    male_votes = male_matrix.sum(axis=0)
    male_max_votes = male_votes.max()
    male_top_figures = male_votes[male_votes == male_max_votes].index.tolist()
    male_top_label = sorted(male_top_figures)[0] if male_top_figures else None

    # Per gruppo femminile: prendi righe solo di annotatori femmine
    female_matrix = pd.DataFrame([dfs_figs[name].loc[idx].values for name in female_annotators], columns=all_figures)
    female_votes = female_matrix.sum(axis=0)
    female_max_votes = female_votes.max()
    female_top_figures = female_votes[female_votes == female_max_votes].index.tolist()
    female_top_label = sorted(female_top_figures)[0] if female_top_figures else None

    results.append({
        'index': idx[2],
        'post_id': idx[0],
        'reply_id': idx[1],
        'male_most_frequent_label': male_top_label,
        'male_votes': male_max_votes,
        'female_most_frequent_label': female_top_label,
        'female_votes': female_max_votes
    })

df_by_gender = pd.DataFrame(results)
print(f"Totale record analizzati: {len(df_by_gender)}")
df_by_gender.head()


Totale record analizzati: 201


,index,post_id,reply_id,male_most_frequent_label,male_votes,female_most_frequent_label,female_votes
0,41,7892378751459500032,16252800000000000000,RHETORICAL QUESTION,2,RHETORICAL QUESTION,2
1,138,6136925553124996096,18189700000000000000,RHETORICAL QUESTION,2,ANALOGY,1
2,139,1796321055434843904,14810100000000000000,CONTEXT SHIFT,2,CONTEXT SHIFT,3
3,154,13772995230818629632,474398000000000000,FALSE ASSERTION,2,FALSE ASSERTION,3
4,2,7155693771356047360,18161900000000000000,OTHER,3,OTHER,2


In [11]:
# Salva solo dati maschili in un file
df_male = df_by_gender[['index', 'post_id', 'reply_id', 'male_most_frequent_label', 'male_votes']]
df_male = df_male.rename(columns={'male_most_frequent_label': 'actual'})

df_male.to_csv('actuals/male_actual_labels.csv', index=False)

# Salva solo dati femminili in un file
df_female = df_by_gender[['index', 'post_id', 'reply_id', 'female_most_frequent_label', 'female_votes']]
df_female = df_female.rename(columns={'female_most_frequent_label': 'actual'})

df_female.to_csv('actuals/female_actual_labels.csv', index=False)


## Generazione: Estrazione etichetta più frequente per ogni record

In [12]:
import pandas as pd

# Lista annotatori e gruppo per genere (modifica se necessario)
genx_annotators = ["Alessandro", "Cristina"]
geny_annotators = ["Michele", "Eliana"]
genz_annotators = ["Michael", "Elisa"]

# Prepara i dataframe indicizzati per ogni annotatore
dfs_figs = {}
for name in sheet_names:
    df_tmp = final_cleaned[name][['post_id', 'reply_id', 'index'] + all_figures].copy()
    dfs_figs[name] = df_tmp.set_index(['post_id', 'reply_id', 'index'])

# Trova record comuni a tutti
common_index = set(dfs_figs[sheet_names[0]].index)
for name in sheet_names[1:]:
    common_index = common_index.intersection(dfs_figs[name].index)
common_index = list(common_index)

results = []

for idx in common_index:
    # Gruppo Gen X
    genx_matrix = pd.DataFrame([dfs_figs[name].loc[idx].values for name in genx_annotators], columns=all_figures)
    genx_votes = genx_matrix.sum(axis=0)
    genx_max_votes = genx_votes.max()
    genx_top_figures = genx_votes[genx_votes == genx_max_votes].index.tolist()
    genx_top_label = sorted(genx_top_figures)[0] if genx_top_figures else None

    # Gruppo Gen Y
    geny_matrix = pd.DataFrame([dfs_figs[name].loc[idx].values for name in geny_annotators], columns=all_figures)
    geny_votes = geny_matrix.sum(axis=0)
    geny_max_votes = geny_votes.max()
    geny_top_figures = geny_votes[geny_votes == geny_max_votes].index.tolist()
    geny_top_label = sorted(geny_top_figures)[0] if geny_top_figures else None

    # Gruppo Gen Z
    genz_matrix = pd.DataFrame([dfs_figs[name].loc[idx].values for name in genz_annotators], columns=all_figures)
    genz_votes = genz_matrix.sum(axis=0)
    genz_max_votes = genz_votes.max()
    genz_top_figures = genz_votes[genz_votes == genz_max_votes].index.tolist()
    genz_top_label = sorted(genz_top_figures)[0] if genz_top_figures else None

    results.append({
        'index': idx[2],
        'post_id': idx[0],
        'reply_id': idx[1],
        'genx_most_frequent_label': genx_top_label,
        'genx_votes': genx_max_votes,
        'geny_most_frequent_label': geny_top_label,
        'geny_votes': geny_max_votes,
        'genz_most_frequent_label': genz_top_label,
        'genz_votes': genz_max_votes,
    })

df_by_generation = pd.DataFrame(results)
print(f"Totale record analizzati: {len(df_by_generation)}")
df_by_generation.head()



Totale record analizzati: 201


,index,post_id,reply_id,genx_most_frequent_label,genx_votes,geny_most_frequent_label,geny_votes,genz_most_frequent_label,genz_votes
0,41,7892378751459500032,16252800000000000000,RHETORICAL QUESTION,2,CONTEXT SHIFT,1,HYPERBOLE,1
1,138,6136925553124996096,18189700000000000000,CONTEXT SHIFT,1,ANALOGY,1,HYPERBOLE,2
2,139,1796321055434843904,14810100000000000000,CONTEXT SHIFT,2,CONTEXT SHIFT,2,CONTEXT SHIFT,1
3,154,13772995230818629632,474398000000000000,FALSE ASSERTION,2,FALSE ASSERTION,1,FALSE ASSERTION,2
4,2,7155693771356047360,18161900000000000000,OTHER,2,OTHER,2,CONTEXT SHIFT,1


In [13]:
# Estrai dati per ogni generazione
df_genx = df_by_generation[['index', 'post_id', 'reply_id', 'genx_most_frequent_label', 'genx_votes']]
df_genx = df_genx.rename(columns={'genx_most_frequent_label': 'actual'})

df_geny = df_by_generation[['index', 'post_id', 'reply_id', 'geny_most_frequent_label', 'geny_votes']]
df_geny = df_geny.rename(columns={'geny_most_frequent_label': 'actual'})

df_genz = df_by_generation[['index', 'post_id', 'reply_id', 'genz_most_frequent_label', 'genz_votes']]
df_genz = df_genz.rename(columns={'genz_most_frequent_label': 'actual'})

# Salva su file CSV separati
df_genx.to_csv('actuals/genx_actual_labels.csv', index=False)
df_geny.to_csv('actuals/geny_actual_labels.csv', index=False)
df_genz.to_csv('actuals/genz_actual_labels.csv', index=False)
